In [1]:
from __future__ import annotations

from typing import List, Union
import numpy as np
import pandas as pd
import yfinance as yf


# ============================================================
# 1) Get price(s) for tickers at specific date(s) for backtest
# ============================================================
def get_prices_on_dates(
    tickers: Union[str, List[str]],
    dates: Union[str, List[str]],
    field_preference: List[str] = ["Adj Close", "Close"],
    method: str = "ffill",     # "exact" | "ffill" | "bfill" | "nearest"
    auto_download_buffer_days: int = 10,
    round_decimals: int = 2,
) -> pd.DataFrame:
    """
    Returns a WIDE table (internal use):
    index = RequestedDate (string)
    columns = tickers + <ticker>__matched_date
    """

    # Normalize tickers
    if isinstance(tickers, str):
        tickers_list = [tickers]
    else:
        tickers_list = list(dict.fromkeys([str(t).strip() for t in tickers if str(t).strip()]))

    # Normalize dates
    if isinstance(dates, str):
        dates_list = [dates]
    else:
        dates_list = [str(d) for d in dates]

    if not tickers_list:
        raise ValueError("tickers kosong.")
    if not dates_list:
        raise ValueError("dates kosong.")

    target_dates = pd.to_datetime(dates_list).tz_localize(None)
    dmin = target_dates.min() - pd.Timedelta(days=auto_download_buffer_days)
    dmax = target_dates.max() + pd.Timedelta(days=auto_download_buffer_days)

    start = dmin.strftime("%Y-%m-%d")
    end = (dmax + pd.Timedelta(days=1)).strftime("%Y-%m-%d")  # safer end

    # Download
    data = yf.download(
        tickers_list,
        start=start,
        end=end,
        auto_adjust=False,
        progress=False,
        group_by="column",
        threads=True,
    )

    if data is None or data.empty:
        raise ValueError("Download data kosong. Cek ticker / koneksi.")

    # Pick Adj Close then Close
    price = None
    if isinstance(data.columns, pd.MultiIndex):
        for f in field_preference:
            if f in data.columns.get_level_values(0):
                price = data[f].copy()
                break
    else:
        price = data.copy()

    if price is None or price.empty:
        raise ValueError("Tidak menemukan kolom harga (Adj Close/Close).")

    price.index = pd.to_datetime(price.index).tz_localize(None)
    idx = price.index  # DatetimeIndex (trading days)

    out = pd.DataFrame(index=target_dates, columns=tickers_list, dtype=float)
    matched = {t: [] for t in tickers_list}

    for d in target_dates:
        # Choose matched trading date
        if method == "exact":
            chosen = d if d in idx else pd.NaT

        elif method == "ffill":
            pos = idx.searchsorted(d, side="right") - 1
            chosen = idx[pos] if pos >= 0 else pd.NaT

        elif method == "bfill":
            pos = idx.searchsorted(d, side="left")
            chosen = idx[pos] if pos < len(idx) else pd.NaT

        elif method == "nearest":
            deltas = (idx - d).to_numpy()
            abs_deltas = np.abs(deltas.astype("timedelta64[ns]").astype(np.int64))
            if abs_deltas.size == 0:
                chosen = pd.NaT
            else:
                min_abs = abs_deltas.min()
                tie_positions = np.where(abs_deltas == min_abs)[0]
                chosen = idx[tie_positions].min()  # tie -> earlier

        else:
            raise ValueError('method harus salah satu: "exact","ffill","bfill","nearest".')

        # Fill prices
        for t in tickers_list:
            if pd.isna(chosen) or t not in price.columns:
                out.loc[d, t] = np.nan
                matched[t].append(pd.NaT)
            else:
                out.loc[d, t] = float(price.loc[chosen, t])
                matched[t].append(chosen)

    # Attach matched date columns
    for t in tickers_list:
        out[f"{t}__matched_date"] = matched[t]

    # Round numeric outputs only
    out = out.round(round_decimals)

    # Set requested dates as index strings
    out.index = pd.Index(dates_list, name="RequestedDate")
    return out


# ============================================================
# 2) Reshape WIDE -> TIDY (ordered by ticker input, not alphabet)
# ============================================================
def reshape_price_table(
    df_wide: pd.DataFrame,
    ticker_order: List[str],
) -> pd.DataFrame:
    """
    Output TIDY table:
    RequestedDate | Ticker | Price | Matched Date
    ordered by ticker_order (NOT alphabetical).
    """
    df = df_wide.copy()

    # index -> column
    df.index.name = "RequestedDate"
    df = df.reset_index()

    # identify columns
    price_cols = [c for c in df.columns if c != "RequestedDate" and not c.endswith("__matched_date")]
    matched_cols = [c for c in df.columns if c.endswith("__matched_date")]

    # melt prices
    price_long = df.melt(
        id_vars="RequestedDate",
        value_vars=price_cols,
        var_name="Ticker",
        value_name="Price",
    )

    # melt matched dates
    matched_long = df.melt(
        id_vars="RequestedDate",
        value_vars=matched_cols,
        var_name="Ticker",
        value_name="Matched Date",
    )
    matched_long["Ticker"] = matched_long["Ticker"].str.replace("__matched_date", "", regex=False)

    # merge
    out = price_long.merge(
        matched_long,
        on=["RequestedDate", "Ticker"],
        how="left",
    )

    # force ticker order
    out["Ticker"] = pd.Categorical(out["Ticker"], categories=ticker_order, ordered=True)

    # nice formatting for matched date
    out["Matched Date"] = pd.to_datetime(out["Matched Date"], errors="coerce").dt.strftime("%Y-%m-%d")

    # sort by date then ticker order
    out = out.sort_values(["RequestedDate", "Ticker"]).reset_index(drop=True)

    return out


# ============================================================
# 3) RUN (NO WIDE OUTPUT PRINTED)
# ============================================================
if __name__ == "__main__":
    # --- set your inputs here
    tickers = ['TMUS', 'SO', 'WMT', 'T', 'PGR', 'NEE', 'PM', 'BSX', 'AVGO', 'FTNT']
    dates = ["2024-12-01"]  # you can put many dates here

    # WIDE is used internally only (not printed)
    df_wide = get_prices_on_dates(
        tickers=tickers,
        dates=dates,
        method="ffill",       # strict backtest (no look-ahead)
        round_decimals=2,
    )

    # TIDY output (printed)
    df_tidy = reshape_price_table(df_wide, ticker_order=tickers)

    print(df_tidy.to_string(index=False))


RequestedDate Ticker  Price Matched Date
   2024-12-01   TMUS 243.15   2024-11-29
   2024-12-01     SO  86.24   2024-11-29
   2024-12-01    WMT  91.42   2024-11-29
   2024-12-01      T  22.17   2024-11-29
   2024-12-01    PGR 263.50   2024-11-29
   2024-12-01    NEE  76.29   2024-11-29
   2024-12-01     PM 127.11   2024-11-29
   2024-12-01    BSX  90.66   2024-11-29
   2024-12-01   AVGO 160.20   2024-11-29
   2024-12-01   FTNT  95.05   2024-11-29
